In [75]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random


# Content-Based Filtering menggunakan Cosine Similarity

## Membuat rekomendasi berdasarkan kriteria
   rekomendasi berdasarkan kriteria menggunakan 4 kriteria<br>
    1.**Halal**<br>
    2.**Area merokok**<br>
    3.**Harga**<br>
    4.**Tempat Parkir**<br>

In [76]:
data ='Downloads/tempat_makan_2022-02-17.csv'
warung = pd.read_csv(data)
print(warung.info())
warung.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747 entries, 0 to 746
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              747 non-null    int64  
 1   Latitude        747 non-null    float64
 2   Longitude       747 non-null    float64
 3   Nama_warung     747 non-null    object 
 4   Alamat          747 non-null    object 
 5   Halal           747 non-null    object 
 6   Area_merokok    747 non-null    object 
 7   Harga           747 non-null    object 
 8   Tempat_parkir   747 non-null    object 
 9   Masakan         746 non-null    object 
 10  Makanan         747 non-null    object 
 11  Jenis_warung    747 non-null    object 
 12  Makan_ditempat  747 non-null    object 
 13  Rating          747 non-null    float64
 14  Review          747 non-null    int64  
 15  Gambar          747 non-null    object 
dtypes: float64(3), int64(2), object(11)
memory usage: 61.3+ KB
None


,ID,Latitude,Longitude,Nama_warung,Alamat,Halal,Area_merokok,Harga,Tempat_parkir,Masakan,Makanan,Jenis_warung,Makan_ditempat,Rating,Review,Gambar
0,1,-7.768875,110.317707,Warung Bakso Babi 08,"Jl. Krandon, Area Sawah, Sidomoyo, Kec. Godean...",non-halal,Ada,Murah,off-street-parking,Indonesia,"Sate,Pork,Bakso",Quick Bites,Dine-in/Takeaway,4.5,52,https://lh5.googleusercontent.com/p/AF1QipNaby...
1,2,-7.765697,110.344304,Rumah Makan 88 / panpan 88,"Jl. Kabupaten KM 1,5 No.98, Nusupan, Trihanggo...",non-halal,Ada,Mahal,on-street-parking,"Chinese,Indonesia,Fusion",Pork,Quick Bites,Dine-in/Takeaway,4.6,671,https://lh5.googleusercontent.com/p/AF1QipMuA7...
2,3,-7.755880,110.383782,Kafe Djawa a.k.a Angkringan Enaknan,"Jalan Kaliurang Km.5,8, Gang Pandega Satya No....",halal,Ada,Murah,on-street-parking,"Jawa,Indonesia,Kafe","Ayam,Daging,Mie,Kopi,Minuman","Kafe,Restoran kasual,Angkringan",Dine-in/Takeaway,4.6,531,https://lh5.googleusercontent.com/p/AF1QipNmmT...
3,4,-7.784784,110.370503,Kali Code Angkringan,"Jl. Prau, Kotabaru, Kec. Gondokusuman, Kota Yo...",halal,Ada,Murah,on-street-parking,"Jawa,Indonesia,Yogyakarta","Sate,Gorengan","Quick Bites,Street Food,Angkringan",Dine-in/Takeaway,4.3,969,https://lh5.googleusercontent.com/p/AF1QipMMru...
4,5,-7.755350,110.376957,Angkringan Kebun,"Jl. Pandega Marta Jl. Pogung Lor No.316, RT.03...",halal,Ada,Murah,on-street-parking,"Jawa,Indonesia,Yogyakarta","Sate,Nasi Goreng,Pecel,Mie,Minuman,Gorengan","Quick Bites,Angkringan",Dine-in/Takeaway,4.9,143,https://lh5.googleusercontent.com/p/AF1QipOu__...


In [77]:
warung['Halal'] = warung['Halal'].str.lower()
warung['Harga'] = warung['Harga'].str.lower()
warung['Area_merokok']= warung['Area_merokok'].str.lower()
warung['Tempat_parkir'] = warung['Tempat_parkir'].str.lower()

In [78]:
warung

,ID,Latitude,Longitude,Nama_warung,Alamat,Halal,Area_merokok,Harga,Tempat_parkir,Masakan,Makanan,Jenis_warung,Makan_ditempat,Rating,Review,Gambar
0,1,-7.768875,110.317707,Warung Bakso Babi 08,"Jl. Krandon, Area Sawah, Sidomoyo, Kec. Godean...",non-halal,ada,murah,off-street-parking,Indonesia,"Sate,Pork,Bakso",Quick Bites,Dine-in/Takeaway,4.5,52,https://lh5.googleusercontent.com/p/AF1QipNaby...
1,2,-7.765697,110.344304,Rumah Makan 88 / panpan 88,"Jl. Kabupaten KM 1,5 No.98, Nusupan, Trihanggo...",non-halal,ada,mahal,on-street-parking,"Chinese,Indonesia,Fusion",Pork,Quick Bites,Dine-in/Takeaway,4.6,671,https://lh5.googleusercontent.com/p/AF1QipMuA7...
2,3,-7.755880,110.383782,Kafe Djawa a.k.a Angkringan Enaknan,"Jalan Kaliurang Km.5,8, Gang Pandega Satya No....",halal,ada,murah,on-street-parking,"Jawa,Indonesia,Kafe","Ayam,Daging,Mie,Kopi,Minuman","Kafe,Restoran kasual,Angkringan",Dine-in/Takeaway,4.6,531,https://lh5.googleusercontent.com/p/AF1QipNmmT...
3,4,-7.784784,110.370503,Kali Code Angkringan,"Jl. Prau, Kotabaru, Kec. Gondokusuman, Kota Yo...",halal,ada,murah,on-street-parking,"Jawa,Indonesia,Yogyakarta","Sate,Gorengan","Quick Bites,Street Food,Angkringan",Dine-in/Takeaway,4.3,969,https://lh5.googleusercontent.com/p/AF1QipMMru...
4,5,-7.755350,110.376957,Angkringan Kebun,"Jl. Pandega Marta Jl. Pogung Lor No.316, RT.03...",halal,ada,murah,on-street-parking,"Jawa,Indonesia,Yogyakarta","Sate,Nasi Goreng,Pecel,Mie,Minuman,Gorengan","Quick Bites,Angkringan",Dine-in/Takeaway,4.9,143,https://lh5.googleusercontent.com/p/AF1QipOu__...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,745,-7.777875,110.345271,Shiki Japanese Food,"Jl. Nusa Indah No.127, Kwarasan, Nogotirto, Ke...",halal,tidak,murah,on-street-parking,"Jepang,Asia","Ayam,Sushi","Restoran kasual,Quick Bites",Dine-in/Takeaway,4.8,114,https://lh5.googleusercontent.com/p/AF1QipMH0V...
743,746,-7.765771,110.405852,WAROENK RAMEN PERUMNAS JOGJA,"Jl. Perumnas No.29, Dabag, Condongcatur, Kec. ...",halal,ada,murah,off-street-parking,"Jepang,Asia","Ayam,Mie,Sushi","Restoran kasual,Quick Bites",Dine-in/Takeaway,4.2,675,https://lh5.googleusercontent.com/p/AF1QipP6J7...
744,747,-7.881513,110.341594,Rock N Roll Sushi Ramen,"Jl. Pramuka, Area Sawah, Trirenggo, Kec. Bantu...",halal,ada,murah,on-street-parking,"Jepang,Asia","Mie,Sushi","Restoran kasual,Quick Bites",Dine-in/Takeaway,4.4,390,https://lh5.googleusercontent.com/p/AF1QipMDN6...
745,748,-7.778105,110.392818,Hakata Ikkousha Ramen Resto,"Ruko Demangan, Kav 2-3, Jl. Demangan Baru No.1...",halal,tidak,mahal,off-street-parking,"Jepang,Asia","Ayam,Daging,Mie,Sushi","Restoran kasual,Restoran Keluarga",Dine-in/Takeaway,4.6,1140,https://lh5.googleusercontent.com/p/AF1QipMqe8...


In [79]:
warung['Halal'] = warung['Halal'].replace('non-halal','non halal')
warung['Tempat_parkir'] = warung['Tempat_parkir'].replace('off-street-parking','off street parking')
warung['Tempat_parkir'] = warung['Tempat_parkir'].replace('on-street-parking','on street parking')

In [80]:
warung['kriteria'] = warung['Halal'].str.cat(
    warung[['Area_merokok','Harga','Tempat_parkir','Halal']],
    sep=' '
)

In [81]:
import re
warung['kriteria'] = warung['kriteria'].apply(lambda x: ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", x).split()))


In [82]:
warung.iloc[230]

ID                                                              233
Latitude                                                   -7.74777
Longitude                                                   110.362
Nama_warung                        Babi Q Resto Khusus Masakan Babi
Alamat            Unnamed Road, 55284, Mlati Dukuh, Sendangadi, ...
Halal                                                     non halal
Area_merokok                                                  tidak
Harga                                                      menengah
Tempat_parkir                                     on street parking
Masakan                              Chinese,Medan,Indonesia,Fusion
Makanan                                                   Sate,Pork
Jenis_warung                            Restoran kasual,Quick Bites
Makan_ditempat                                     Dine-in/Takeaway
Rating                                                          4.5
Review                                          

In [83]:
warung.to_csv('warung.csv', index = False)

In [84]:
test1 =pd.read_csv('warung.csv')
test1

,ID,Latitude,Longitude,Nama_warung,Alamat,Halal,Area_merokok,Harga,Tempat_parkir,Masakan,Makanan,Jenis_warung,Makan_ditempat,Rating,Review,Gambar,kriteria
0,1,-7.768875,110.317707,Warung Bakso Babi 08,"Jl. Krandon, Area Sawah, Sidomoyo, Kec. Godean...",non halal,ada,murah,off street parking,Indonesia,"Sate,Pork,Bakso",Quick Bites,Dine-in/Takeaway,4.5,52,https://lh5.googleusercontent.com/p/AF1QipNaby...,non halal ada murah off street parking non halal
1,2,-7.765697,110.344304,Rumah Makan 88 / panpan 88,"Jl. Kabupaten KM 1,5 No.98, Nusupan, Trihanggo...",non halal,ada,mahal,on street parking,"Chinese,Indonesia,Fusion",Pork,Quick Bites,Dine-in/Takeaway,4.6,671,https://lh5.googleusercontent.com/p/AF1QipMuA7...,non halal ada mahal on street parking non halal
2,3,-7.755880,110.383782,Kafe Djawa a.k.a Angkringan Enaknan,"Jalan Kaliurang Km.5,8, Gang Pandega Satya No....",halal,ada,murah,on street parking,"Jawa,Indonesia,Kafe","Ayam,Daging,Mie,Kopi,Minuman","Kafe,Restoran kasual,Angkringan",Dine-in/Takeaway,4.6,531,https://lh5.googleusercontent.com/p/AF1QipNmmT...,halal ada murah on street parking halal
3,4,-7.784784,110.370503,Kali Code Angkringan,"Jl. Prau, Kotabaru, Kec. Gondokusuman, Kota Yo...",halal,ada,murah,on street parking,"Jawa,Indonesia,Yogyakarta","Sate,Gorengan","Quick Bites,Street Food,Angkringan",Dine-in/Takeaway,4.3,969,https://lh5.googleusercontent.com/p/AF1QipMMru...,halal ada murah on street parking halal
4,5,-7.755350,110.376956,Angkringan Kebun,"Jl. Pandega Marta Jl. Pogung Lor No.316, RT.03...",halal,ada,murah,on street parking,"Jawa,Indonesia,Yogyakarta","Sate,Nasi Goreng,Pecel,Mie,Minuman,Gorengan","Quick Bites,Angkringan",Dine-in/Takeaway,4.9,143,https://lh5.googleusercontent.com/p/AF1QipOu__...,halal ada murah on street parking halal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,745,-7.777875,110.345272,Shiki Japanese Food,"Jl. Nusa Indah No.127, Kwarasan, Nogotirto, Ke...",halal,tidak,murah,on street parking,"Jepang,Asia","Ayam,Sushi","Restoran kasual,Quick Bites",Dine-in/Takeaway,4.8,114,https://lh5.googleusercontent.com/p/AF1QipMH0V...,halal tidak murah on street parking halal
743,746,-7.765771,110.405852,WAROENK RAMEN PERUMNAS JOGJA,"Jl. Perumnas No.29, Dabag, Condongcatur, Kec. ...",halal,ada,murah,off street parking,"Jepang,Asia","Ayam,Mie,Sushi","Restoran kasual,Quick Bites",Dine-in/Takeaway,4.2,675,https://lh5.googleusercontent.com/p/AF1QipP6J7...,halal ada murah off street parking halal
744,747,-7.881513,110.341594,Rock N Roll Sushi Ramen,"Jl. Pramuka, Area Sawah, Trirenggo, Kec. Bantu...",halal,ada,murah,on street parking,"Jepang,Asia","Mie,Sushi","Restoran kasual,Quick Bites",Dine-in/Takeaway,4.4,390,https://lh5.googleusercontent.com/p/AF1QipMDN6...,halal ada murah on street parking halal
745,748,-7.778105,110.392818,Hakata Ikkousha Ramen Resto,"Ruko Demangan, Kav 2-3, Jl. Demangan Baru No.1...",halal,tidak,mahal,off street parking,"Jepang,Asia","Ayam,Daging,Mie,Sushi","Restoran kasual,Restoran Keluarga",Dine-in/Takeaway,4.6,1140,https://lh5.googleusercontent.com/p/AF1QipMqe8...,halal tidak mahal off street parking halal


In [85]:
#count kriteria
model = CountVectorizer(
    tokenizer = lambda i: i.split(' '), # -> cari split karakter yang unik
    analyzer = 'word'
)
matrix_kriteria = model.fit_transform(warung['kriteria'])
tipe_kriteria = model.get_feature_names()
jumlah_kriteria = len(tipe_kriteria)
event_kriteria = matrix_kriteria.toarray()

In [86]:
print (tipe_kriteria)

['ada', 'halal', 'mahal', 'menengah', 'murah', 'non', 'off', 'on', 'parking', 'street', 'tidak']


In [87]:
print (jumlah_kriteria)

11


In [88]:
print(event_kriteria)

[[1 2 0 ... 1 1 0]
 [1 2 1 ... 1 1 0]
 [1 2 0 ... 1 1 0]
 ...
 [1 2 0 ... 1 1 0]
 [0 2 1 ... 1 1 1]
 [0 2 1 ... 1 1 1]]


In [89]:
print(event_kriteria[230])
print(event_kriteria[234])

[0 2 0 1 0 2 0 1 1 1 1]
[1 2 0 1 0 2 0 1 1 1 0]


In [90]:
print(event_kriteria[1])
print(event_kriteria[2])

[1 2 1 0 0 2 0 1 1 1 0]
[1 2 0 0 1 0 0 1 1 1 0]


In [91]:
print(matrix_kriteria[230])

  (0, 5)	2
  (0, 1)	2
  (0, 9)	1
  (0, 8)	1
  (0, 7)	1
  (0, 10)	1
  (0, 3)	1


In [92]:
score = cosine_similarity(matrix_kriteria)
score

array([[1.        , 0.84615385, 0.73960026, ..., 0.73960026, 0.64715023,
        0.64715023],
       [0.84615385, 1.        , 0.73960026, ..., 0.73960026, 0.64715023,
        0.64715023],
       [0.73960026, 0.73960026, 1.        , ..., 1.        , 0.66666667,
        0.66666667],
       ...,
       [0.73960026, 0.73960026, 1.        , ..., 1.        , 0.66666667,
        0.66666667],
       [0.64715023, 0.64715023, 0.66666667, ..., 0.66666667, 1.        ,
        1.        ],
       [0.64715023, 0.64715023, 0.66666667, ..., 0.66666667, 1.        ,
        1.        ]])

## Membuat Cosine Similarity Score

In [93]:
#cosine similarity
score = cosine_similarity(matrix_kriteria)
score

array([[1.        , 0.84615385, 0.73960026, ..., 0.73960026, 0.64715023,
        0.64715023],
       [0.84615385, 1.        , 0.73960026, ..., 0.73960026, 0.64715023,
        0.64715023],
       [0.73960026, 0.73960026, 1.        , ..., 1.        , 0.66666667,
        0.66666667],
       ...,
       [0.73960026, 0.73960026, 1.        , ..., 1.        , 0.66666667,
        0.66666667],
       [0.64715023, 0.64715023, 0.66666667, ..., 0.66666667, 1.        ,
        1.        ],
       [0.64715023, 0.64715023, 0.66666667, ..., 0.66666667, 1.        ,
        1.        ]])

## Membuat Machine Learning Prdiction
membuat dummy dataframe untuk prosses kategori data yand ada didalam kolom kriteria

In [94]:
warung_dummy = pd.get_dummies(
    warung,
    columns = ['Halal','Area_merokok','Harga','Tempat_parkir'], 
)
display(warung_dummy.head())

,ID,Latitude,Longitude,Nama_warung,Alamat,Masakan,Makanan,Jenis_warung,Makan_ditempat,Rating,...,kriteria,Halal_halal,Halal_non halal,Area_merokok_ada,Area_merokok_tidak,Harga_mahal,Harga_menengah,Harga_murah,Tempat_parkir_off street parking,Tempat_parkir_on street parking
0,1,-7.768875,110.317707,Warung Bakso Babi 08,"Jl. Krandon, Area Sawah, Sidomoyo, Kec. Godean...",Indonesia,"Sate,Pork,Bakso",Quick Bites,Dine-in/Takeaway,4.5,...,non halal ada murah off street parking non halal,0,1,1,0,0,0,1,1,0
1,2,-7.765697,110.344304,Rumah Makan 88 / panpan 88,"Jl. Kabupaten KM 1,5 No.98, Nusupan, Trihanggo...","Chinese,Indonesia,Fusion",Pork,Quick Bites,Dine-in/Takeaway,4.6,...,non halal ada mahal on street parking non halal,0,1,1,0,1,0,0,0,1
2,3,-7.755880,110.383782,Kafe Djawa a.k.a Angkringan Enaknan,"Jalan Kaliurang Km.5,8, Gang Pandega Satya No....","Jawa,Indonesia,Kafe","Ayam,Daging,Mie,Kopi,Minuman","Kafe,Restoran kasual,Angkringan",Dine-in/Takeaway,4.6,...,halal ada murah on street parking halal,1,0,1,0,0,0,1,0,1
3,4,-7.784784,110.370503,Kali Code Angkringan,"Jl. Prau, Kotabaru, Kec. Gondokusuman, Kota Yo...","Jawa,Indonesia,Yogyakarta","Sate,Gorengan","Quick Bites,Street Food,Angkringan",Dine-in/Takeaway,4.3,...,halal ada murah on street parking halal,1,0,1,0,0,0,1,0,1
4,5,-7.755350,110.376957,Angkringan Kebun,"Jl. Pandega Marta Jl. Pogung Lor No.316, RT.03...","Jawa,Indonesia,Yogyakarta","Sate,Nasi Goreng,Pecel,Mie,Minuman,Gorengan","Quick Bites,Angkringan",Dine-in/Takeaway,4.9,...,halal ada murah on street parking halal,1,0,1,0,0,0,1,0,1


menentukan variabel independen dan dependent

In [154]:
#label encoder
from sklearn.preprocessing import LabelEncoder
category_col = ['Halal','Area_merokok','Harga','Tempat_parkir']
labelencoder = LabelEncoder()
warung['kriteria_cat'] = labelencoder.fit_transform(warung['kriteria'])
warung.head()


,ID,Latitude,Longitude,Nama_warung,Alamat,Halal,Area_merokok,Harga,Tempat_parkir,Masakan,Makanan,Jenis_warung,Makan_ditempat,Rating,Review,Gambar,kriteria,kriteria_cat
0,1,-7.768875,110.317707,Warung Bakso Babi 08,"Jl. Krandon, Area Sawah, Sidomoyo, Kec. Godean...",non halal,ada,murah,off street parking,Indonesia,"Sate,Pork,Bakso",Quick Bites,Dine-in/Takeaway,4.5,52,https://lh5.googleusercontent.com/p/AF1QipNaby...,non halal ada murah off street parking non halal,16
1,2,-7.765697,110.344304,Rumah Makan 88 / panpan 88,"Jl. Kabupaten KM 1,5 No.98, Nusupan, Trihanggo...",non halal,ada,mahal,on street parking,"Chinese,Indonesia,Fusion",Pork,Quick Bites,Dine-in/Takeaway,4.6,671,https://lh5.googleusercontent.com/p/AF1QipMuA7...,non halal ada mahal on street parking non halal,13
2,3,-7.755880,110.383782,Kafe Djawa a.k.a Angkringan Enaknan,"Jalan Kaliurang Km.5,8, Gang Pandega Satya No....",halal,ada,murah,on street parking,"Jawa,Indonesia,Kafe","Ayam,Daging,Mie,Kopi,Minuman","Kafe,Restoran kasual,Angkringan",Dine-in/Takeaway,4.6,531,https://lh5.googleusercontent.com/p/AF1QipNmmT...,halal ada murah on street parking halal,5
3,4,-7.784784,110.370503,Kali Code Angkringan,"Jl. Prau, Kotabaru, Kec. Gondokusuman, Kota Yo...",halal,ada,murah,on street parking,"Jawa,Indonesia,Yogyakarta","Sate,Gorengan","Quick Bites,Street Food,Angkringan",Dine-in/Takeaway,4.3,969,https://lh5.googleusercontent.com/p/AF1QipMMru...,halal ada murah on street parking halal,5
4,5,-7.755350,110.376957,Angkringan Kebun,"Jl. Pandega Marta Jl. Pogung Lor No.316, RT.03...",halal,ada,murah,on street parking,"Jawa,Indonesia,Yogyakarta","Sate,Nasi Goreng,Pecel,Mie,Minuman,Gorengan","Quick Bites,Angkringan",Dine-in/Takeaway,4.9,143,https://lh5.googleusercontent.com/p/AF1QipOu__...,halal ada murah on street parking halal,5


In [155]:
x = warung_dummy.drop(
    ['ID','Latitude','Longitude','Nama_warung','Alamat','Masakan','Makanan',
    'Makan_ditempat','kriteria','Jenis_warung','Rating','Review','Gambar'], axis=1
)
display(x.head())

,Halal_halal,Halal_non halal,Area_merokok_ada,Area_merokok_tidak,Harga_mahal,Harga_menengah,Harga_murah,Tempat_parkir_off street parking,Tempat_parkir_on street parking
0,0,1,1,0,0,0,1,1,0
1,0,1,1,0,1,0,0,0,1
2,1,0,1,0,0,0,1,0,1
3,1,0,1,0,0,0,1,0,1
4,1,0,1,0,0,0,1,0,1


In [156]:
display(x)

,Halal_halal,Halal_non halal,Area_merokok_ada,Area_merokok_tidak,Harga_mahal,Harga_menengah,Harga_murah,Tempat_parkir_off street parking,Tempat_parkir_on street parking
0,0,1,1,0,0,0,1,1,0
1,0,1,1,0,1,0,0,0,1
2,1,0,1,0,0,0,1,0,1
3,1,0,1,0,0,0,1,0,1
4,1,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
742,1,0,0,1,0,0,1,0,1
743,1,0,1,0,0,0,1,1,0
744,1,0,1,0,0,0,1,0,1
745,1,0,0,1,1,0,0,1,0


In [157]:
y = warung_dummy['Nama_warung']
display(y.head())

0                   Warung Bakso Babi 08
1             Rumah Makan 88 / panpan 88
2    Kafe Djawa a.k.a Angkringan Enaknan
3                   Kali Code Angkringan
4                       Angkringan Kebun
Name: Nama_warung, dtype: object

In [158]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(x, y, test_size = 0.67, random_state = 5)

In [68]:
print(X_train.shape)
X_train

(246, 9)


,Halal_halal,Halal_non halal,Area_merokok_ada,Area_merokok_tidak,Harga_mahal,Harga_menengah,Harga_murah,Tempat_parkir_off street parking,Tempat_parkir_on street parking
570,1,0,1,0,0,0,1,0,1
213,1,0,1,0,0,0,1,0,1
583,1,0,1,0,0,1,0,1,0
83,1,0,0,1,0,1,0,1,0
92,1,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
73,1,0,1,0,0,1,0,1,0
400,1,0,1,0,0,0,1,0,1
118,1,0,0,1,0,1,0,0,1
701,1,0,1,0,0,1,0,1,0


In [69]:
print(X_test.shape)

(501, 9)


In [70]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [160]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state =1234)
rf_clf.fit(X_train, y_train)

# scoring model
print('Accuracy = {}%'.format(round(rf_clf.score(X_train, y_train) * 100, 2)))
# print(rf_clf.score(x, y))
# prediction
# print(rf_clf.predict([[1,0,1,0,0,0,1,1,0]])[0])

y_pred = rf_clf.predict(X_test)

Accuracy = 7.32%


<i>Machine Learning</i> memprediksi nama warung berdasarkan kriteria berikut :<br>
<br>
 1.**Halal** = **halal**<br>
 2.**Area Merokok** = **Ada**<br>
 3.**Harga** = **Murah**<br>
 4.**Tempat Parkir** = **off-street-parking**<br>

In [172]:
# logistic regression model
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver = 'liblinear', multi_class = 'auto')
log_reg.fit(x, y)

# scoring model
print('Accuracy = {}%'.format(round(log_reg.score(x, y) * 100, 2)))

# prediction
print(log_reg.predict([[1,0,1,0,0,0,1,1,0]])[0])

Accuracy = 2.81%
Mugi Mojo Kitchen


In [80]:
# kernel svm model
from sklearn.svm import SVC
svm = SVC(kernel = 'rbf', gamma = 0.5)
svm.fit(x, y)

# scoring model
print('Accuracy = {}%'.format(round(svm.score(x, y) * 100, 2)))

# prediction
print(svm.predict([[1,0,1,0,0,0,1,1,0]])[0])

Accuracy = 2.81%
kitchen Oriental Seafood


In [103]:
# naive bayes model
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# scoring model
print('Accuracy = {}%'.format(round(gnb.score(x, y) * 100, 2)))

# prediction
print(gnb.predict([[1,0,1,0,0,0,1,1,0]])[0])

Accuracy = 2.41%
Aneka Bubur "Assalam"


In [102]:
# decision tree classifier
from sklearn.tree import DecisionTreeClassifier
dct_clf = DecisionTreeClassifier(criterion = 'entropy')
dct_clf.fit(X_train, y_train)

# scoring model
print('Accuracy = {}%'.format(round(dct_clf.score(x, y) * 100, 2)))

# prediction
print(dct_clf.predict([[0,1,1,0,0,0,1,1,0]])[0])

Accuracy = 2.41%
Aneka Bubur "Assalam"


In [159]:
# random forest model
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state =0)
rf_clf.fit(X_test, y_test)

# prediction
print(rf_clf.predict([[0,1,1,0,1,0,0,1,0]])[0])

# scoring model
print('Accuracy = {}%'.format(round(rf_clf.score(x, y) * 100, 2)))

Kong Kong Chinese & Seafood
Accuracy = 2.41%


In [84]:
display(warung[warung['Nama_warung'] == 'Mugi Mojo Kitchen'])
display(warung[warung['Nama_warung'] == 'kitchen Oriental Seafood'])
display(warung[warung['Nama_warung'] == 'Aneka Bubur "Assalam"'])
display(warung[warung['Nama_warung'] == 'Woko Wader mangut'])

,ID,Latitude,Longitude,Nama_warung,Alamat,Halal,Area_merokok,Harga,Tempat_parkir,Masakan,Makanan,Jenis_warung,Makan_ditempat,Rating,Review,Gambar,kriteria
258,261,-7.724268,110.380645,Mugi Mojo Kitchen,"Jl. Sadewa No.414, Tegal Rejo, Sariharjo, Kec....",halal,ada,murah,off street parking,"Jawa,Indonesia","Tahu Tek,Lontong","Restoran kasual,Quick Bites",Dine-in/Takeaway,5.0,6,https://lh5.googleusercontent.com/p/AF1QipPWEX...,halal ada murah off street parking


,ID,Latitude,Longitude,Nama_warung,Alamat,Halal,Area_merokok,Harga,Tempat_parkir,Masakan,Makanan,Jenis_warung,Makan_ditempat,Rating,Review,Gambar,kriteria
690,693,-7.750924,110.410149,kitchen Oriental Seafood,"Jl. Super Raya No.169-177, Dero, Condongcatur,...",halal,ada,murah,off street parking,"Chinese,Indonesia,Asia,Fusion","Ayam,Ikan,Seafood,Hot Plate","Restoran kasual,Quick Bites",Dine-in/Takeaway,4.7,136,https://lh5.googleusercontent.com/p/AF1QipOWt7...,halal ada murah off street parking


,ID,Latitude,Longitude,Nama_warung,Alamat,Halal,Area_merokok,Harga,Tempat_parkir,Masakan,Makanan,Jenis_warung,Makan_ditempat,Rating,Review,Gambar,kriteria
622,625,-7.760825,110.380502,"Aneka Bubur ""Assalam""","Karang Wuni, Caturtunggal, Kec. Depok, Kabupat...",halal,ada,murah,off street parking,Indonesia,Bubur Kacang Ijo,Quick Bites,Dine-in/Takeaway,4.1,25,https://lh5.googleusercontent.com/p/AF1QipOq3x...,halal ada murah off street parking


,ID,Latitude,Longitude,Nama_warung,Alamat,Halal,Area_merokok,Harga,Tempat_parkir,Masakan,Makanan,Jenis_warung,Makan_ditempat,Rating,Review,Gambar,kriteria
590,593,-7.831751,110.463049,Woko Wader mangut,"Onggoparum, Srimulyo, Kec. Piyungan, Bantul",halal,ada,murah,off street parking,"Jawa,Indonesia,Yogyakarta","Ikan,Mangut Lele","Restoran kasual,Quick Bites",Dine-in/Takeaway,4.5,133,https://lh5.googleusercontent.com/p/AF1QipPWXX...,halal ada murah off street parking


<h2> Membuat <i>recommendation System</i> berdasarkan kriteria</h2> <br>
    dari hasil prediksi yang didapat penelti memilih <i>Random Forest Classifer</i> peneliti ingin mendapatkan 5 rekoemdasi yang meiliki <i>cosine similarity</i> > 84%

In [173]:
# test model 1, 0, 0, 1, 0, 0, 1, 0, 1
warung_fav = rf_clf.predict([[0,1,1,0,0,0,1,1,0]])[0]

display(warung[warung['Nama_warung'] == warung_fav])

# take the index from warung_fav
index_fav = warung[warung['Nama_warung'] == warung_fav].index.values[0]
# print(index_fav)

# list all restuarants + cosine similarity score
all_warung = list(enumerate(score[index_fav]))
# print(all_warung)

# show similar restaurant, sorted by score
warung_similar = sorted(
    all_warung,
    key = lambda i: i[1],
    reverse = True
)
# print(resto_similar)

,ID,Latitude,Longitude,Nama_warung,Alamat,Halal,Area_merokok,Harga,Tempat_parkir,Masakan,Makanan,Jenis_warung,Makan_ditempat,Rating,Review,Gambar,kriteria,kriteria_cat
233,236,-7.773472,110.411075,Depot babi YOE,"Unnamed Road, Kledokan, Caturtunggal, Kec. Dep...",non halal,ada,murah,on street parking,"Jawa,Indonesia","Sate,Pork,Nasi Goreng,Mie",Quick Bites,Dine-in/Takeaway,4.7,6,https://lh5.googleusercontent.com/p/AF1QipOx3r...,non halal ada murah on street parking non halal,17


In [186]:
warung.iloc[248]

ID                                                              251
Latitude                                                   -7.78632
Longitude                                                   110.302
Nama_warung                DJOMED br. Tampubolon ( Rm daging babi )
Alamat            Jalan Pandawa RT.05/RW.18, Gancakan Wetan, Sid...
Halal                                                     non halal
Area_merokok                                                    ada
Harga                                                      menengah
Tempat_parkir                                    off street parking
Masakan                              Chinese,Medan,Indonesia,Fusion
Makanan                            Pork,Nasi Goreng,Soto,Sop,Bakmie
Jenis_warung                            Restoran kasual,Quick Bites
Makan_ditempat                                     Dine-in/Takeaway
Rating                                                          4.7
Review                                          

In [175]:
all_warung

[(0, 0.9230769230769231),
 (1, 0.9230769230769229),
 (2, 0.8320502943378435),
 (3, 0.8320502943378435),
 (4, 0.8320502943378435),
 (5, 0.8320502943378435),
 (6, 0.8320502943378435),
 (7, 0.554700196225229),
 (8, 0.554700196225229),
 (9, 0.554700196225229),
 (10, 0.554700196225229),
 (11, 0.554700196225229),
 (12, 0.554700196225229),
 (13, 0.6471502289294339),
 (14, 0.7396002616336387),
 (15, 0.6471502289294339),
 (16, 0.7396002616336387),
 (17, 0.6471502289294339),
 (18, 0.7396002616336387),
 (19, 0.846153846153846),
 (20, 0.6471502289294339),
 (21, 0.6471502289294339),
 (22, 0.554700196225229),
 (23, 0.554700196225229),
 (24, 0.6471502289294339),
 (25, 0.7396002616336387),
 (26, 0.6471502289294339),
 (27, 0.8320502943378435),
 (28, 0.8320502943378435),
 (29, 0.8320502943378435),
 (30, 0.8320502943378435),
 (31, 0.8320502943378435),
 (32, 0.8320502943378435),
 (33, 0.6471502289294339),
 (34, 0.7396002616336387),
 (35, 0.7396002616336387),
 (36, 0.7396002616336387),
 (37, 0.647150228929

In [176]:
warung_similar

[(233, 1.0),
 (242, 1.0),
 (244, 1.0),
 (245, 1.0),
 (246, 1.0),
 (0, 0.9230769230769231),
 (1, 0.9230769230769229),
 (232, 0.9230769230769229),
 (234, 0.9230769230769229),
 (237, 0.9230769230769229),
 (240, 0.9230769230769229),
 (241, 0.9230769230769229),
 (243, 0.9230769230769229),
 (247, 0.9230769230769229),
 (305, 0.9230769230769229),
 (341, 0.9230769230769229),
 (19, 0.846153846153846),
 (230, 0.846153846153846),
 (235, 0.846153846153846),
 (236, 0.846153846153846),
 (238, 0.846153846153846),
 (239, 0.846153846153846),
 (248, 0.846153846153846),
 (2, 0.8320502943378435),
 (3, 0.8320502943378435),
 (4, 0.8320502943378435),
 (5, 0.8320502943378435),
 (6, 0.8320502943378435),
 (27, 0.8320502943378435),
 (28, 0.8320502943378435),
 (29, 0.8320502943378435),
 (30, 0.8320502943378435),
 (31, 0.8320502943378435),
 (32, 0.8320502943378435),
 (67, 0.8320502943378435),
 (68, 0.8320502943378435),
 (69, 0.8320502943378435),
 (86, 0.8320502943378435),
 (88, 0.8320502943378435),
 (90, 0.83205029

In [247]:
# list all warung filter by cosine similarity score > 75%
warung_recom = []
for i in warung_similar:
    if i[1] >= 0.84:
        warung_recom.append(i)
    else:
        pass

# show 5 datas randomly
import random
#random.sample(range(23), 10)
for i in range(23):
    rekomendasi = random.sample(warung_recom, 10)
#rekomendasi = random.choices(list(warung_recom), k = 10)

for i in rekomendasi:
    print('Warung = {} | Halal = {} | Area Merokok = {} | Harga = {} | Tempat parkir = {}'.format(
        warung.iloc[i[0]]['Nama_warung'],
        warung.iloc[i[0]]['Halal'],
        warung.iloc[i[0]]['Area_merokok'],
        warung.iloc[i[0]]['Harga'],
        warung.iloc[i[0]]['Tempat_parkir'])
    )

    

Warung = Warung Makan Bu Kadek Khas Bali | Halal = non halal | Area Merokok = ada | Harga = menengah | Tempat parkir = off street parking
Warung = Warung Bakso Babi 08 | Halal = non halal | Area Merokok = ada | Harga = murah | Tempat parkir = off street parking
Warung = Babi Q Resto Khusus Masakan Babi | Halal = non halal | Area Merokok = tidak | Harga = menengah | Tempat parkir = on street parking
Warung = Bak & Mie - Spesial Babi Geprek & Mie Babi | Halal = non halal | Area Merokok = ada | Harga = menengah | Tempat parkir = on street parking
Warung = Kedai Piggies | Halal = non halal | Area Merokok = ada | Harga = menengah | Tempat parkir = on street parking
Warung = Babi Panggang Racing Koh JW | Halal = non halal | Area Merokok = tidak | Harga = mahal | Tempat parkir = on street parking
Warung = Warung Babi Si Lae | Halal = non halal | Area Merokok = ada | Harga = murah | Tempat parkir = on street parking
Warung = bakso dan sate Babi pak Har | Halal = non halal | Area Merokok = ada 

In [236]:
all_lines = f1.readlines()

NameError: name 'f1' is not defined

In [198]:
#rekomendasi = random.choices(warung_recom, k = 5)
list_rekom = []
for i in warung_recom[:5]:
    ambil_rekom = {}
    ambil_rekom['Nama_warung'] = warung.iloc[i[0]]['Nama_warung'],
    ambil_rekom['Halal'] = warung.iloc[i[0]]['Halal'],
    ambil_rekom['Area_merokok'] = warung.iloc[i[0]]['Area_merokok'],
    ambil_rekom['Harga'] = warung.iloc[i[0]]['Harga'],
    ambil_rekom['Tempat_parkir'] = warung.iloc[i[0]]['Tempat_parkir'],
    ambil_rekom['Latitude'] = warung.iloc[i[0]]['Latitude'],
    ambil_rekom['Longitude'] = warung.iloc[i[0]]['Longitude']
    
    list_rekom.append(ambil_rekom)
   
    

In [179]:
list_rekom

[{'Nama_warung': ('Depot babi YOE',),
  'Halal': ('non halal',),
  'Area_merokok': ('ada',),
  'Harga': ('murah',),
  'Tempat_parkir': ('on street parking',),
  'Latitude': (-7.7734719,),
  'Longitude': 110.4110748},
 {'Nama_warung': ('Gemah Ripah - Special Masakan Daging Babi',),
  'Halal': ('non halal',),
  'Area_merokok': ('ada',),
  'Harga': ('murah',),
  'Tempat_parkir': ('on street parking',),
  'Latitude': (-7.7846564,),
  'Longitude': 110.3789477},
 {'Nama_warung': ('Nasi Goreng Babi Kranggan',),
  'Halal': ('non halal',),
  'Area_merokok': ('ada',),
  'Harga': ('murah',),
  'Tempat_parkir': ('on street parking',),
  'Latitude': (-7.7816487,),
  'Longitude': 110.3646812},
 {'Nama_warung': ('Warung Babi Si Lae',),
  'Halal': ('non halal',),
  'Area_merokok': ('ada',),
  'Harga': ('murah',),
  'Tempat_parkir': ('on street parking',),
  'Latitude': (-7.7854563,),
  'Longitude': 110.3787934},
 {'Nama_warung': ('bakso dan sate Babi pak Har',),
  'Halal': ('non halal',),
  'Area_mero

In [180]:
ambil_rekom

{'Nama_warung': ('bakso dan sate Babi pak Har',),
 'Halal': ('non halal',),
 'Area_merokok': ('ada',),
 'Harga': ('murah',),
 'Tempat_parkir': ('on street parking',),
 'Latitude': (-7.7727407,),
 'Longitude': 110.3264233}

In [188]:
warung_recom = []
for i in warung_similar:
    if i[1] >= 0.84:
        warung_recom.append(i)
    else:
        pass

In [189]:
warung_recom

[(233, 1.0),
 (242, 1.0),
 (244, 1.0),
 (245, 1.0),
 (246, 1.0),
 (0, 0.9230769230769231),
 (1, 0.9230769230769229),
 (232, 0.9230769230769229),
 (234, 0.9230769230769229),
 (237, 0.9230769230769229),
 (240, 0.9230769230769229),
 (241, 0.9230769230769229),
 (243, 0.9230769230769229),
 (247, 0.9230769230769229),
 (305, 0.9230769230769229),
 (341, 0.9230769230769229),
 (19, 0.846153846153846),
 (230, 0.846153846153846),
 (235, 0.846153846153846),
 (236, 0.846153846153846),
 (238, 0.846153846153846),
 (239, 0.846153846153846),
 (248, 0.846153846153846)]

In [201]:
display(warung[warung['Halal'] == 'non_halal'])

,ID,Latitude,Longitude,Nama_warung,Alamat,Halal,Area_merokok,Harga,Tempat_parkir,Masakan,Makanan,Jenis_warung,Makan_ditempat,Rating,Review,Gambar,kriteria
0,1,-7.768875,110.317707,Warung Bakso Babi 08,"Jl. Krandon, Area Sawah, Sidomoyo, Kec. Godean...",non_halal,Ada,Murah,off_street_parking,Indonesia,"Sate,Pork,Bakso",Quick Bites,Dine-in/Takeaway,4.5,52,AF1QipNaby7OGc0DBcboM9zR8cMXbZ3KHGIKA5JculpO=w...,non_halal Ada Murah off_street_parking
1,2,-7.765697,110.344304,Rumah Makan 88 / panpan 88,"Jl. Kabupaten KM 1,5 No.98, Nusupan, Trihanggo...",non_halal,Ada,Mahal,on_street_parking,"Chinese,Indonesia,Fusion",Pork,Quick Bites,Dine-in/Takeaway,4.6,671,AF1QipMuA78dB9x1zyraLsyshQ4_2rk1j95XNJYgPuWZ=s...,non_halal Ada Mahal on_street_parking
19,20,-7.818566,110.364528,Mediterranea Restaurant,"Jl. Tirtodipuran No. 24 A Mantrijeron, Yogyaka...",non_halal,Ada,Mahal,off_street_parking,"Eropa,Mediterania,Perancis,Western,Italia,Dess...","Daging,Burger,Pizza,Kopi,Minuman","Restoran kasual,Diner,Wine Bar",Dine-in/Takeaway,4.7,5106,https://lh3.ggpht.com/p/AF1QipN2MevJ2aAr369_dj...,non_halal Ada Mahal off_street_parking
230,233,-7.747771,110.362017,Babi Q Resto Khusus Masakan Babi,"Unnamed Road, 55284, Mlati Dukuh, Sendangadi, ...",non_halal,Tidak,Menengah,on_street_parking,"Chinese,Medan,Indonesia,Fusion","Sate,Pork","Restoran kasual,Quick Bites",Dine-in/Takeaway,4.5,21,AF1QipNeL2MVmIJuVrARXyJiTo7MOe0nXUhK2Mny6dI=s5...,non_halal Tidak Menengah on_street_parking
232,235,-7.773496,110.411448,Bak & Mie - Spesial Babi Geprek & Mie Babi,"Kompleks Pertokoan Babarsari No.37a, Kledokan,...",non_halal,Ada,Menengah,on_street_parking,"Chinese,Indonesia,Fusion","Pork,Mie",Quick Bites,Dine-in/Takeaway,4.5,367,AF1QipPfxx5JxO9HKiSR2vYn3kHMZvFWt8dLC3E5QWSy=s...,non_halal Ada Menengah on_street_parking
233,236,-7.773472,110.411075,Depot babi YOE,"Unnamed Road, Kledokan, Caturtunggal, Kec. Dep...",non_halal,Ada,Murah,on_street_parking,"Jawa,Indonesia","Sate,Pork,Nasi Goreng,Mie",Quick Bites,Dine-in/Takeaway,4.7,6,AF1QipOx3r7IPwGGWsT52u52N-tFCChLEKgpgKm1LmMi=s...,non_halal Ada Murah on_street_parking
234,237,-7.772725,110.391792,Pothz,"Jl. STM Pembangunan No.14, Santren, Caturtungg...",non_halal,Ada,Menengah,on_street_parking,"Eropa,Indonesia,Western,Fusion","Pork,Nasi Goreng,Burger",Quick Bites,Dine-in/Takeaway,4.6,1281,AF1QipN9xxaISIV5QwMIWsJM6FMuTVWXOEuBn0ycuWS4=s...,non_halal Ada Menengah on_street_parking
235,238,-7.770619,110.397099,Babi Panggang Racing Koh JW,"Jl. Garuda Jl. Pringwulung No.162A, Pringwulun...",non_halal,Tidak,Mahal,on_street_parking,"Chinese,Indonesia","Sate,Pork,Bakso",Quick Bites,Cuma Takeaway,4.5,243,AF1QipO-N3ndA4TpekwXCFwZQgSqL_te2pJ7p6uOqk63=s...,non_halal Tidak Mahal on_street_parking
236,239,-7.762582,110.397731,Lapo Laetta,"Selatan Hartono Mall, Jl. Kaliwaru No.mor 812,...",non_halal,Ada,Menengah,off_street_parking,"Medan,Indonesia","Pork,Mie,Sop,Nasi Campur","Restoran kasual,Quick Bites,Restoran Keluarga",Dine-in/Takeaway,4.5,61,AF1QipM-KlqeesxQ7NLJ8kRFXdOmPMX60SM-GglGkOjR=s...,non_halal Ada Menengah off_street_parking
237,240,-7.782023,110.362401,Kedai Piggies,"Cokrodiningratan, Kec. Jetis, Kota Yogyakarta",non_halal,Ada,Menengah,on_street_parking,"Chinese,Indonesia","Pork,Nasi Goreng,Mie,Mie Ayam,Nasi Campur",Quick Bites,Dine-in/Takeaway,4.2,40,AF1QipNRfSo8FRcjHbWtW1mprNohQp6l3WlG16YrDAry=s...,non_halal Ada Menengah on_street_parking


<h2> Simpan <i>random forest classifier model</i></h2>

In [190]:
import joblib
joblib.dump(rf_clf,'model_warung')

['model_warung']

In [249]:
def recommendation(prediksi):
    #recommended_warung = []
    warung_fav = rf_clf.predict([prediksi])[0]
    matrix_kriteria = model.fit_transform(warung['kriteria'])
    score = cosine_similarity(matrix_kriteria)
    
    index_fav = warung[warung['Nama_warung'] == warung_fav].index.values[0]
    # list all restuarants + cosine similarity score
    all_warung = list(enumerate(score[index_fav]))
    # show similar restaurant, sorted by score
    warung_similar = sorted(
    all_warung,
    key = lambda i: i[1],
    reverse = True
    )
    warung_recom = []
    for i in warung_similar:
        if i[1] > 0.84:
            warung_recom.append(i)
        else:
            pass

    # show 5 datas randomly
    list_rekom = []
    rekomendasi = random.choices(warung_recom, k = 5)
    
    for i in rekomendasi:
        
     print('Warung = {} | Halal = {} | Area Merokok = {} | Harga = {} | Tempat parkir = {}'.format(
        warung.iloc[i[0]]['Nama_warung'],
        warung.iloc[i[0]]['Halal'],
        warung.iloc[i[0]]['Area_merokok'],
        warung.iloc[i[0]]['Harga'],
        warung.iloc[i[0]]['Tempat_parkir'])
        )
    return #recommended_warung

In [250]:
Data = [0,1,1,0,0,0,1,0,1]
recommendation(data)

ValueError: could not convert string to float: 'Downloads/tempat_makan_2022-02-17.csv'

In [251]:
Data = [0,1,1,0,0,0,1,0,1]
#prediksi = rf_clf.predict([fitur])[0]
#warung_fav = prediksi
# print(data)
recommendation(data)

ValueError: could not convert string to float: 'Downloads/tempat_makan_2022-02-17.csv'

In [ ]:
class recommederSystem :
    def __init__(self, data,content_col)
        self.warung = pd.read_csv(data)
        self.c
    def fit(self):
    def recommend(self):